In [50]:
import fastf1 as ff1
import pandas as pd
import time

In [51]:
# Config cache
ff1.Cache.enable_cache('.cache')

# Empty the cache
ff1.Cache.clear_cache('.cache')

In [52]:
years = range(2018, 2026) # 2026 is excluded. 2017 is not included because the data missing telemetry data

In [53]:
# # Info on the session
# session = ff1.get_session(2021, 'Bahrain Grand Prix', 'R')
# print(session.event)

In [54]:
# # Info on the year 2021
# schedule = ff1.get_event_schedule(2021)
# schedule

# General info

In [55]:
# schedule = ff1.get_event_schedule(2025)
# schedule

## Number of races for year

In [56]:
# total_races_from_2000 = 0
# for year in years:
#     try:
#         schedule = ff1.get_event_schedule(year)
#         print(f'{year}: {len(schedule)}')
#         total_races_from_2000 += len(schedule)
#     except:
#         print(f'{year}: No data')

# print(f'Total races in period 2000-2025: {total_races_from_2000}')


## Info aggregated by pilot and laps

In [57]:
# session = ff1.get_session(2024, 'French Grand Prix', 'R')
# session.load()
# session.laps


In [58]:
# print(session.laps.Driver.value_counts())

# Cicle all the laps

In [59]:
years = range(2018, 2026) # 2026 is excluded

In [60]:
def load_session(year, event_name, max_attempts=5):
    attempts = 0
    while attempts < max_attempts:
        try:
            session = ff1.get_session(year, event_name, 'R')
            return  session # Esci dalla funzione se il caricamento ha successo
        except Exception as e:
            attempts += 1
            print(f"Tentativo {attempts} fallito per {year} - {event_name}: {e}")
            time.sleep(10)
    print(f"\nCaricamento fallito per {session.event} dopo {max_attempts} tentativi.")
    return None
    

In [ ]:
import warnings
import io
import sys
import logging
from contextlib import redirect_stdout, redirect_stderr

number_of_laps = 0
missing_data = []
warnings_log = {}  # Dictionary to store warnings by (year, event_name)
all_warning_lines = []  # List to store all lines containing "warning"

# Get the root logger to capture all logging output
root_logger = logging.getLogger()
original_level = root_logger.level
original_handlers = root_logger.handlers.copy()

for year in years:
    # Get the schedule for the year
    scheduled = ff1.get_event_schedule(year)

    # Select scheduled event that are races
    scheduled_race = scheduled[scheduled['Session5'] == 'Race'] # There are session that are not races

    # Get the event name
    event_names = scheduled_race['EventName'].values

    # Cicle through the events
    for event_name in event_names:
        print(f'--> {year}: {event_name}')

        session = load_session(year, event_name)
        if session is None:
            missing_data.append((year, event_name))
            continue

        # Sometimes the data is not available
        try:
            # Setup log capture
            log_capture = io.StringIO()
            log_handler = logging.StreamHandler(log_capture)
            log_handler.setLevel(logging.WARNING)  # Capture WARNING and higher
            log_formatter = logging.Formatter('%(levelname)s - %(name)s - %(message)s')
            log_handler.setFormatter(log_formatter)
            
            # Temporarily modify logger to capture all warnings
            root_logger.setLevel(logging.WARNING)
            root_logger.handlers = [log_handler]
            
            # Also capture stdout/stderr for any direct prints
            stdout_capture = io.StringIO()
            stderr_capture = io.StringIO()
            
            # Load session with all captures in place
            with redirect_stdout(stdout_capture), redirect_stderr(stderr_capture), warnings.catch_warnings(record=True) as caught_warnings:
                warnings.simplefilter("always")
                try:
                    session.load(laps=True)
                except Exception as e:
                    missing_data.append((year, event_name))
                    warnings_log[(year, event_name)] = {'error': str(e)}
            # Restore original logging configuration
            root_logger.setLevel(original_level)
            root_logger.handlers = original_handlers.copy()
            
            # Get captured outputs
            log_output = log_capture.getvalue()
            stdout_output = stdout_capture.getvalue()
            stderr_output = stderr_capture.getvalue()
            warning_messages = [str(w.message) for w in caught_warnings]
            
            # Check for warnings in log output
            log_lines = log_output.split('\n')
            for line in log_lines:
                if 'warning' in line.lower() or 'error' in line.lower():
                    all_warning_lines.append(f"{year} {event_name}: {line}")
            
            # Also check stdout and stderr
            for line in stdout_output.split('\n'):
                if 'warning' in line.lower() or 'error' in line.lower():
                    all_warning_lines.append(f"{year} {event_name}: {line}")
            
            for line in stderr_output.split('\n'):
                if 'warning' in line.lower() or 'error' in line.lower():
                    all_warning_lines.append(f"{year} {event_name}: {line}")
            
            # Store warnings if any were found
            if all_warning_lines:
                warnings_log[(year, event_name)] = {
                    'log_output': log_output,
                    'stdout': stdout_output, 
                    'stderr': stderr_output,
                    'python_warnings': warning_messages
                }
                print(f"  Captured warnings for {year} {event_name}")

            # Get the total number of laps in the session
            total_session_laps = session.laps.shape[0]
            number_of_laps += total_session_laps

            # Get drivers names
            drivers = session.laps.Driver.unique()

            # cicle through drivers
            for driver in drivers:
                pass
                    
        except Exception as e:
            missing_data.append((year, event_name))
            warnings_log[(year, event_name)] = {'error': str(e)}
            continue
        
    ff1.Cache.clear_cache('.cache')

    # Save all warning lines to a file
    if all_warning_lines:
        with open('warning_lines.txt', 'w') as f:
            with open('warning_lines.txt', 'a') as f_append:
                f_append.write('\n'.join(all_warning_lines))

        print(f"\nTotal warning lines: {len(all_warning_lines)}")
        print(f"Warning lines saved to 'warning_lines.txt'")


# For debugging: show a sample of the captured warnings
if all_warning_lines:
    print("\nSample of captured warnings:")
    for line in all_warning_lines[:5]:  # Show first 5 warnings
        print(line)

core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!


--> 2018: Australian Grand Prix


req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to cache!
core           INFO 	Processing timing data.

  Captured warnings for 2018 Australian Grand Prix
--> 2018: Bahrain Grand Prix


_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
_api        WARNING 	Failed to align laps for drivers: ['3', '33']
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for timing_app_data. Loading data...
_api           INFO 	Fetching timing app data...
req            INFO 	Data has been written to cache!
core           INFO 	Processing timing data...
core        W

...

req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
_api        WARNING 	Driver 63: Encountered 1 timing

  Captured warnings for 2020 Austrian Grand Prix
--> 2020: Styrian Grand Prix


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
_api        WARNING 	Failed to align laps for driver

  Captured warnings for 2020 Styrian Grand Prix
--> 2020: Hungarian Grand Prix


req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	

In [ ]:
print(f'Total number of laps in period 2018-2025: {number_of_laps}')


NameError: name 'number_of_laps' is not defined